# Creating a Basic Agent with OpenAI Agents SDK

This notebook demonstrates how to create and run a simple agent using the OpenAI Agents SDK.

## Creating a Basic Agent

We'll create a simple agent with basic instructions.

In [10]:
# Add the project root to the Python path
import sys
from pathlib import Path
import os
from dotenv import load_dotenv
from agents import set_default_openai_key

sys.path.insert(0, "..")

load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")
set_default_openai_key(openai_api_key)

In [14]:
# Define a basic agent
from agents import Agent, Runner

# Create the basic agent 
basic_agent = Agent(
    name="basic_agent",
    instructions="You are a helpful agent that provides concise and accurate information."
)

# Get  output from the agent 
output = await Runner.run(basic_agent, "What's the capital of the USA?")
print(output.final_output)



The capital of the USA is Washington, D.C.


## Running the Agent

Now let's run the agent with a simple question.

In [ ]:
# Run the agent with a question
result = await AgentRunner.run(basic_agent, ["What's the capital of the USA?"])

# Print the agent's response
print(result.agent_output)

## Adding a Simple Tool

Now let's create an agent with a basic tool function.

In [ ]:
from agents.tool import function_tool

# Define a simple calculator tool
@function_tool
def calculate(expression: str) -> str:
    """
    Evaluates a mathematical expression and returns the result.
    
    Args:
        expression: A string representing a mathematical expression (e.g., "2 + 2")
        
    Returns:
        The result of the calculation as a string
    """
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

In [ ]:
# Create an agent with the calculator tool
calculator_agent = Agent(
    name="calculator_agent",
    instructions="You are a helpful assistant that can perform calculations. When asked about math problems, use the calculate tool.",
    tools=[calculate]
)

In [ ]:
# Run the agent with a calculation request
result = await AgentRunner.run(calculator_agent, ["What is 125 * 37?"])

# Print the agent's response
print(result.agent_output)

## Using Structured Output

Let's create an agent that returns structured data using Pydantic models.

In [ ]:
from pydantic import BaseModel

class WeatherAnswer(BaseModel):
    location: str
    temperature_c: float
    summary: str

In [ ]:
# Create an agent with structured output
structured_agent = Agent(
    name="structured_weather_agent",
    instructions="You provide weather information in a structured format. Use the final_output tool with the WeatherAnswer schema.",
    output_type=WeatherAnswer
)

In [ ]:
# Run the agent with a weather question
result = await AgentRunner.run(structured_agent, ["What's the weather like in Paris?"])

# Print the structured output
print(f"Location: {result.agent_output.location}")
print(f"Temperature: {result.agent_output.temperature_c}°C")
print(f"Summary: {result.agent_output.summary}")

## Using Context

Let's create an agent that uses context to maintain state between interactions.

In [ ]:
from agents.run_context import AgentContextWrapper

class MyContext:
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.interaction_count = 0

@function_tool
def greet_user(context: AgentContextWrapper[MyContext], greeting: str) -> str:
    context.agent_context.interaction_count += 1
    return f"Hello {context.agent_context.user_id}, you said: {greeting}. This is interaction #{context.agent_context.interaction_count}"

In [ ]:
# Create an agent with context
context_agent = Agent(
    name="context_agent",
    instructions="You are a friendly agent that greets users by name and keeps track of interactions.",
    context_type=MyContext,
    tools=[greet_user]
)

In [ ]:
# Create a context object
my_context = MyContext(user_id="Alice")

# Run the agent with context
result = await AgentRunner.run(
    context_agent,
    input=["Hello agent!"],
    context=my_context
)

print(result.agent_output)

In [ ]:
# Run it again with the same context to see the interaction count increase
result = await AgentRunner.run(
    context_agent,
    input=["Hi again!"],
    context=my_context
)

print(result.agent_output)